In [1]:
import tensorflow as tf  
import numpy as np       
import pandas as pd       
from matplotlib import pyplot as plt   
import seaborn as sns  
import time          
from tensorflow.keras.callbacks import History, ReduceLROnPlateau, TensorBoard       
from tensorflow.keras import metrics          
from sklearn.metrics import confusion_matrix          
import pickle
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, multiply, add, Dense, Conv2D, GlobalMaxPooling2D, Activation
from tensorflow.keras import backend as K
import platform
import os
from tensorflow.keras.layers import GlobalAveragePooling2D, Multiply, Dense
from keras.layers import GlobalAveragePooling2D, Dense, Multiply
import platform
import os

In [2]:
# Display Settings
pd.options.display.max_columns = None
pd.options.display.max_rows = None
sns.set()

In [3]:
# System Information Print the Python version
print("Python version: ", platform.python_version())
print("TensorFlow version: ", tf.__version__)
print("Current working directory: ", os.getcwd())
!uname -a
!nvidia-smi

Python version:  3.11.7
TensorFlow version:  2.15.0
Current working directory:  /root/.jupyter/张彤/注意力机制
Linux w3q2ulc9.vm 5.15.0-60-generic #66-Ubuntu SMP Fri Jan 20 14:29:49 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux
Sat Oct 26 21:08:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        On  | 00000000:09:00.0 Off |                  Off |
|  0%   24C    P8               9

In [4]:
# Hyperparameter Settings
EPOCHS = 50
IMAGE_SIZE = (224, 224)
IMAGE_PATH = "../data"
LEARNING_RATE = 1e-4
BATCH_SIZE = 64

In [5]:
# Load Dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    IMAGE_PATH,
    validation_split=0.2,
    subset="training",    
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

vaild_ds = tf.keras.utils.image_dataset_from_directory(
    IMAGE_PATH,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

Found 14080 files belonging to 100 classes.
Using 11264 files for training.


2024-10-26 21:09:13.848903: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-26 21:09:13.849398: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-26 21:09:14.168137: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Found 14080 files belonging to 100 classes.
Using 2816 files for validation.


In [6]:
# Image Normalization
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

def normalize_image(image):
    return (image - mean_tensor) / std_tensor


In [7]:
# Data Augmentation
train_image_augment = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1 / 255.0),
    tf.keras.layers.RandomRotation(factor=0.2),
    tf.keras.layers.RandomFlip(),
])

valid_image_augment = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1 / 255.0),
])

In [8]:
# Input Processing
def process_train_input(images, labels):
    return train_image_augment(images), labels

def process_valid_input(images, labels):
    return valid_image_augment(images), labels


In [9]:
# Data Preprocessing
def convert_types_and_encode(x, y):
    y = tf.cast(y, tf.int32)  
    y_one_hot = tf.one_hot(y, 100)  
    return x, y_one_hot

In [10]:
# Apply Data Preprocessing
train_ds = train_ds.map(convert_types_and_encode)
train_ds = train_ds.map(process_train_input, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

vaild_ds = vaild_ds.map(convert_types_and_encode)
vaild_ds = vaild_ds.map(process_valid_input, num_parallel_calls=tf.data.AUTOTUNE)
vaild_ds = vaild_ds.prefetch(tf.data.AUTOTUNE)

In [11]:
from keras.layers import GlobalAveragePooling2D, Dense, Multiply

# Define the SE layer
def se_layer(input_tensor, reduction_ratio=16):
    channel = input_tensor.shape[-1]
    # Squeeze operation
    se = GlobalAveragePooling2D()(input_tensor)
    se = Dense(channel // reduction_ratio, activation='relu')(se)
    se = Dense(channel, activation='sigmoid')(se)
    # Reshape and scale the input
    se = tf.reshape(se, [-1, 1, 1, channel])
    return Multiply()([input_tensor, se])

# Create MobileNet base model
base_model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
feature_maps = base_model.output

# Add SE layer
se_feature_maps = se_layer(feature_maps)

# Add global average pooling
x = GlobalAveragePooling2D()(se_feature_maps)

# Add fully connected layers
x = Dense(1024, activation='relu')(x)
outputs = Dense(100, activation='softmax')(x)

# Create the final model
model = tf.keras.Model(inputs=base_model.input, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [12]:
# Compile Model
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
loss_fn = tf.keras.losses.CategoricalCrossentropy()
metrics = [
    tf.keras.metrics.CategoricalAccuracy(),
    tf.keras.metrics.Precision(),
    tf.keras.metrics.Recall(),
]

model.compile(optimizer=optimizer, loss=loss_fn, metrics=metrics)

In [13]:
# Train Model
log_dir = "../Running result/SE/SE"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=2, min_lr=1e-8)

history = model.fit(train_ds, epochs=EPOCHS, validation_data=vaild_ds, callbacks=[lr_scheduler, tensorboard_callback])

Epoch 1/50


2024-10-26 21:09:30.026446: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-10-26 21:09:30.411625: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-26 21:09:31.692746: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f2b1da4c870 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-10-26 21:09:31.692837: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-10-26 21:09:31.692852: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-10-26 21:09:31.729631: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1729948172.011184    1664 device_comp

176/176 [==============================] - 37s 88ms/step - loss: 2.7326 - categorical_accuracy: 0.4195 - precision: 0.8827 - recall: 0.1537 - val_loss: 1.6132 - val_categorical_accuracy: 0.5774 - val_precision: 0.8250 - val_recall: 0.3533 - lr: 1.0000e-04
Epoch 2/50
176/176 [==============================] - 14s 76ms/step - loss: 0.9005 - categorical_accuracy: 0.7550 - precision: 0.9023 - recall: 0.6251 - val_loss: 0.8698 - val_categorical_accuracy: 0.7500 - val_precision: 0.8691 - val_recall: 0.6438 - lr: 1.0000e-04
Epoch 3/50
176/176 [==============================] - 14s 77ms/step - loss: 0.5441 - categorical_accuracy: 0.8467 - precision: 0.9227 - recall: 0.7757 - val_loss: 0.7235 - val_categorical_accuracy: 0.7802 - val_precision: 0.8520 - val_recall: 0.7177 - lr: 1.0000e-04
Epoch 4/50
176/176 [==============================] - 14s 78ms/step - loss: 0.3815 - categorical_accuracy: 0.8924 - precision: 0.9430 - recall: 0.8413 - val_loss: 0.5233 - val_categorical_accuracy: 0.8462 - val

In [14]:
import pandas as pd  # Import Pandas library
import os

# Define save path
model_dir = "../Running result/SE/SE"
excel_file_path = os.path.join(model_dir, "SE.xlsx")  # Path to save the Excel file
model_file_path = os.path.join(model_dir, "SE.h5")  # Path to save the model

# Check if the save directory exists; create it if it does not
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Save training history to an Excel file
history_df = pd.DataFrame(history.history)  
history_df.to_excel(excel_file_path, index=False)
print(f"Training history saved to: {excel_file_path}")

# Save the model to the specified path
model.save(model_file_path)
print(f"Model saved to: {model_file_path}")


Training history saved to: ../Running result/SE/SE/SE.xlsx


/root/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved to: ../Running result/SE/SE/SE.h5
